In [1]:
import django
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'mysite.settings'
django.setup()

In [2]:
from imdb.models import *
from datetime import date
from imdb.model_methodsv2 import *
from django.db.models import *
from django.db import connection

In [ ]:
create_actor()

In [ ]:
create_movie()

In [ ]:
create_rating()

In [ ]:
create_movie_cast()

In [ ]:
max_actor_movies()

In [ ]:
get_remuneration_stats(1)

In [ ]:
get_min_age_of_cast_in_movie()

In [ ]:
get_movies_that_doesnot_have_heroine()

In [ ]:
get_movies_more_than_ten_female()

In [ ]:
get_movie_contains_title()

In [ ]:
diff_between_male_and_female()

In [ ]:
get_movie_contains_title()

In [ ]:
avg_remuneration_by_actors()

In [ ]:
diff_between_male_and_female()

In [ ]:
no_of_actors_same_month()

In [ ]:
no_of_actors_same_year()

In [ ]:
no_of_actors_same_name()

# DB access optimization

# select_related

In [ ]:
len(connection.queries)

In [ ]:
mr=MovieRating.objects.get(id=21)
m=mr.movie#takes 2 hits

In [ ]:
len(connection.queries)

In [ ]:
mr1=MovieRating.objects.select_related('movie').get(id=21)#takes 1 hit

In [ ]:
len(connection.queries)

In [ ]:
connection.queries[-3:]

# prefetch_related

In [5]:
len(connection.queries)

4

In [4]:
m=Movie.objects.filter(id__in=[1,2,3])#takes 1 hit
for i in m:
    print(i.movie_ratings.all())#takes 3hits

<QuerySet [<MovieRating: MovieRating object (1)>, <MovieRating: MovieRating object (2)>, <MovieRating: MovieRating object (3)>, <MovieRating: MovieRating object (4)>, <MovieRating: MovieRating object (5)>]>
<QuerySet [<MovieRating: MovieRating object (6)>, <MovieRating: MovieRating object (7)>, <MovieRating: MovieRating object (8)>, <MovieRating: MovieRating object (9)>, <MovieRating: MovieRating object (10)>]>
<QuerySet [<MovieRating: MovieRating object (11)>, <MovieRating: MovieRating object (12)>, <MovieRating: MovieRating object (13)>, <MovieRating: MovieRating object (14)>, <MovieRating: MovieRating object (15)>]>


In [ ]:
m[1].movie_ratings.all()

In [ ]:
len(connection.queries)

In [6]:
connection.queries[-4:]

[{'sql': 'SELECT "imdb_movie"."id", "imdb_movie"."title", "imdb_movie"."date_of_release" FROM "imdb_movie" WHERE "imdb_movie"."id" IN (1, 2, 3)',
  'time': '0.004'},
 {'sql': 'SELECT "imdb_movierating"."id", "imdb_movierating"."movie_id", "imdb_movierating"."rating", "imdb_movierating"."no_of_ratings" FROM "imdb_movierating" WHERE "imdb_movierating"."movie_id" = 1  LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "imdb_movierating"."id", "imdb_movierating"."movie_id", "imdb_movierating"."rating", "imdb_movierating"."no_of_ratings" FROM "imdb_movierating" WHERE "imdb_movierating"."movie_id" = 2  LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "imdb_movierating"."id", "imdb_movierating"."movie_id", "imdb_movierating"."rating", "imdb_movierating"."no_of_ratings" FROM "imdb_movierating" WHERE "imdb_movierating"."movie_id" = 3  LIMIT 21',
  'time': '0.000'}]

In [7]:
len(connection.queries)

4

In [8]:
m=Movie.objects.prefetch_related('movie_ratings').filter(id__in=[1,2,3])#takes 1 hit
for i in m:
    print(i.movie_ratings.all())#takes 1 hit

<QuerySet [<MovieRating: MovieRating object (1)>, <MovieRating: MovieRating object (2)>, <MovieRating: MovieRating object (3)>, <MovieRating: MovieRating object (4)>, <MovieRating: MovieRating object (5)>]>
<QuerySet [<MovieRating: MovieRating object (6)>, <MovieRating: MovieRating object (7)>, <MovieRating: MovieRating object (8)>, <MovieRating: MovieRating object (9)>, <MovieRating: MovieRating object (10)>]>
<QuerySet [<MovieRating: MovieRating object (11)>, <MovieRating: MovieRating object (12)>, <MovieRating: MovieRating object (13)>, <MovieRating: MovieRating object (14)>, <MovieRating: MovieRating object (15)>]>


In [9]:
len(connection.queries)

6

In [10]:
connection.queries[-2:]

[{'sql': 'SELECT "imdb_movie"."id", "imdb_movie"."title", "imdb_movie"."date_of_release" FROM "imdb_movie" WHERE "imdb_movie"."id" IN (1, 2, 3)',
  'time': '0.000'},
 {'sql': 'SELECT "imdb_movierating"."id", "imdb_movierating"."movie_id", "imdb_movierating"."rating", "imdb_movierating"."no_of_ratings" FROM "imdb_movierating" WHERE "imdb_movierating"."movie_id" IN (1, 2, 3)',
  'time': '0.000'}]

In [ ]:
MovieCast.objects.prefetch_related('movie__actors')

In [ ]:
MovieCast.objects.select_related('movie').prefetch_related('movie__actors')

In [ ]:
m1=MovieCast.objects.prefetch_related('movie','cast')

In [ ]:
m1

In [ ]:
connection.queries[-5:]

In [ ]:
m.values()

In [ ]:
m.filter(movie_ratings__rating=3)

In [ ]:
len(connection.queries)

In [ ]:
m=Movie.objects.prefetch_related('actors')

In [ ]:
m[0].movie_ratings.all()

In [ ]:
connection.queries[-2:]

In [103]:
non_prefetched = m.prefetch_related(None)

In [ ]:
Movie.objects.prefetch_related('actors')

In [119]:
len(connection.queries)

70

In [ ]:
m=Movie.objects.prefetch_related(Prefetch('actors'))

In [ ]:
for i in m:
    print(i.actors.all())

In [120]:
connection.queries[-2:]

[{'sql': 'SELECT "imdb_moviecast"."id", "imdb_moviecast"."movie_id", "imdb_moviecast"."cast_id", "imdb_moviecast"."role", "imdb_moviecast"."remuneration_in_usd", "imdb_movie"."id", "imdb_movie"."title", "imdb_movie"."date_of_release" FROM "imdb_moviecast" INNER JOIN "imdb_movie" ON ("imdb_moviecast"."movie_id" = "imdb_movie"."id")',
  'time': '0.001'},
 {'sql': 'SELECT "imdb_movierating"."id", "imdb_movierating"."movie_id", "imdb_movierating"."rating", "imdb_movierating"."no_of_ratings" FROM "imdb_movierating" WHERE "imdb_movierating"."movie_id" IN (1, 2, 3, 4, 5)',
  'time': '0.000'}]

In [117]:
m=MovieCast.objects.select_related('movie').prefetch_related(Prefetch('movie__movie_ratings',queryset=MovieRating.objects.select_related('movie')))

In [118]:
for i in m:
    print(i.movie.movie_ratings.all())

<QuerySet [<MovieRating: MovieRating object (1)>, <MovieRating: MovieRating object (2)>, <MovieRating: MovieRating object (3)>, <MovieRating: MovieRating object (4)>, <MovieRating: MovieRating object (5)>]>
<QuerySet [<MovieRating: MovieRating object (1)>, <MovieRating: MovieRating object (2)>, <MovieRating: MovieRating object (3)>, <MovieRating: MovieRating object (4)>, <MovieRating: MovieRating object (5)>]>
<QuerySet [<MovieRating: MovieRating object (6)>, <MovieRating: MovieRating object (7)>, <MovieRating: MovieRating object (8)>, <MovieRating: MovieRating object (9)>, <MovieRating: MovieRating object (10)>]>
<QuerySet [<MovieRating: MovieRating object (6)>, <MovieRating: MovieRating object (7)>, <MovieRating: MovieRating object (8)>, <MovieRating: MovieRating object (9)>, <MovieRating: MovieRating object (10)>]>
<QuerySet [<MovieRating: MovieRating object (11)>, <MovieRating: MovieRating object (12)>, <MovieRating: MovieRating object (13)>, <MovieRating: MovieRating object (14)>,

In [35]:
for i in m:
    print(i.movie.movie_ratings.all())

<QuerySet [<MovieRating: MovieRating object (1)>, <MovieRating: MovieRating object (2)>, <MovieRating: MovieRating object (3)>, <MovieRating: MovieRating object (4)>, <MovieRating: MovieRating object (5)>]>
<QuerySet [<MovieRating: MovieRating object (1)>, <MovieRating: MovieRating object (2)>, <MovieRating: MovieRating object (3)>, <MovieRating: MovieRating object (4)>, <MovieRating: MovieRating object (5)>]>
<QuerySet [<MovieRating: MovieRating object (6)>, <MovieRating: MovieRating object (7)>, <MovieRating: MovieRating object (8)>, <MovieRating: MovieRating object (9)>, <MovieRating: MovieRating object (10)>]>
<QuerySet [<MovieRating: MovieRating object (6)>, <MovieRating: MovieRating object (7)>, <MovieRating: MovieRating object (8)>, <MovieRating: MovieRating object (9)>, <MovieRating: MovieRating object (10)>]>
<QuerySet [<MovieRating: MovieRating object (11)>, <MovieRating: MovieRating object (12)>, <MovieRating: MovieRating object (13)>, <MovieRating: MovieRating object (14)>,

In [104]:
m=MovieCast.objects.prefetch_related(Prefetch('movie__movie_ratings'))

In [95]:
m=MovieCast.objects.filter(id__in=[1,2,3])

In [96]:
len(connection.queries)

58

In [90]:
result = MovieCast.objects.prefetch_related(Prefetch('movie',queryset=m,to_attr='first_films'))

In [94]:
connection.queries[-2:]

[{'sql': 'SELECT "imdb_moviecast"."id", "imdb_moviecast"."movie_id", "imdb_moviecast"."cast_id", "imdb_moviecast"."role", "imdb_moviecast"."remuneration_in_usd" FROM "imdb_moviecast"',
  'time': '0.000'},
 {'sql': 'SELECT "imdb_moviecast"."id", "imdb_moviecast"."movie_id", "imdb_moviecast"."cast_id", "imdb_moviecast"."role", "imdb_moviecast"."remuneration_in_usd" FROM "imdb_moviecast" WHERE ("imdb_moviecast"."id" IN (1, 2, 3) AND "imdb_moviecast"."id" IN (1, 2, 3, 4, 5))',
  'time': '0.000'}]

In [92]:
for item in result:
    print(type(item.first_films))

<class 'imdb.models.MovieCast'>
<class 'imdb.models.MovieCast'>
<class 'imdb.models.MovieCast'>
<class 'imdb.models.MovieCast'>
<class 'imdb.models.MovieCast'>
<class 'NoneType'>
<class 'NoneType'>


In [59]:
MovieCast.objects.count()


7

In [60]:
Movie.objects.count()

5

In [78]:
first_films

NameError: name 'first_films' is not defined

In [87]:
Movie.objects.values('id','title')

<QuerySet [{'id': 1, 'title': 'Endgame'}, {'id': 2, 'title': 'Inception'}, {'id': 3, 'title': 'Avatar'}, {'id': 4, 'title': 'Titanic'}, {'id': 5, 'title': 'Clash of titans'}]>